In [1]:
import sys
sys.path.append('/home/wassal/trust-wassal/')
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from geomloss import SamplesLoss
from trust.utils.models.resnet import ResNet18
from trust.utils.models.resnet import ResNet50
import random
import math
from torch.autograd import Variable
from trust.utils.custom_dataset_medmnist import load_biodataset_custom
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import numpy as np
from torch.utils.data import Subset
from trust.utils.utils import SubsetWithTargets

In [4]:

# Load the CIFAR10 dataset
'''transform = transforms.Compose(
    [
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    #Get an initial set of 100 data points
initial_indices = torch.randperm(len(trainset))[:100]
initial_dataset = torch.utils.data.Subset(trainset, initial_indices)
initial_dataloader = DataLoader(dataset=initial_dataset, batch_size=10, shuffle=True)

# Randomly select 1000 unlabelled datapoints
unlabelled_indices = torch.randperm(len(trainset))[:1000]
unlabeled_dataset = torch.utils.data.Subset(trainset, unlabelled_indices)

# Filter the dataset to include only two classes, say, classes 0 and 1
labelled_indices = [i for i, (img, label) in enumerate(trainset) if label in [0, 1]]

# From the filtered dataset, randomly select 100 datapoints
labelled_indices = torch.randperm(len(labelled_indices))[:100]
query_dataset = torch.utils.data.Subset(trainset, labelled_indices)
'''

#trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#load the dataset in the class imbalance setting
feature = "classimb"
datadir = '/home/wassal/trust-wassal/tutorials/All_Wassal/data' #contains the npz file of the data_name dataset listed below
dataset_name = 'pneumoniamnist'

split_cfg = {
             "per_class_train":{0:100,1:5}, 
             "per_class_val":{0:20,1:20}, 
             "per_class_lake":{0:1050,1:53},
             "per_class_test":{0:200,1:200},
             "sel_cls_idx":[1], 
             "per_imbclass_train":{0:100,1:5}, 
             "per_imbclass_val":{0:20,1:20}, 
             "per_imbclass_lake":{0:1050,1:53},
             "per_imbclass_test":{0:200,1:200}}

print("split_cfg:",split_cfg)

trainset, val_set, test_set, lake_set, sel_cls_idx, num_cls = load_biodataset_custom(datadir, dataset_name, feature, split_cfg, False, False)
    
print(sel_cls_idx, num_cls)

#create a dataset that merges trainset and val_set into a dataset of class target and non_target
from torch.utils.data import Dataset, ConcatDataset

class BinaryClassificationDataset(Dataset):
    def __init__(self, original_dataset, sel_cls_idx):
        self.original_dataset = original_dataset
        self.sel_cls_idx = sel_cls_idx

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        data, target = self.original_dataset[idx]

        # Convert the target into binary
        if target == self.sel_cls_idx:
            target = 0
        else:
            target = 1
        
        return data, target

# Create binary datasets
binary_trainset = BinaryClassificationDataset(trainset, sel_cls_idx[0])
binary_val_set = BinaryClassificationDataset(val_set, sel_cls_idx[0])

# Merge trainset and val_set
targetset = ConcatDataset([binary_trainset, binary_val_set])


split_cfg: {'per_class_train': {0: 100, 1: 5}, 'per_class_val': {0: 20, 1: 20}, 'per_class_lake': {0: 1050, 1: 53}, 'per_class_test': {0: 200, 1: 200}, 'sel_cls_idx': [1], 'per_imbclass_train': {0: 100, 1: 5}, 'per_imbclass_val': {0: 20, 1: 20}, 'per_imbclass_lake': {0: 1050, 1: 53}, 'per_imbclass_test': {0: 200, 1: 200}}
Using downloaded and verified file: /home/wassal/trust-wassal/tutorials/All_Wassal/data/pneumoniamnist.npz
Using downloaded and verified file: /home/wassal/trust-wassal/tutorials/All_Wassal/data/pneumoniamnist.npz
pneumoniamnist Custom dataset stats: Train size:  105 Val size:  40 Lake size:  1103
[1] 2


In [ ]:
device = "cuda:"+str(0) if torch.cuda.is_available() else "cpu"
def model_eval_loss(data_loader, model, criterion):
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()
                
def create_model(name, num_cls, device, embedding_type):
    if name == 'ResNet18':
        if embedding_type == "gradients":
            model = ResNet18(num_cls)
        else:
            model = models.resnet18()
    elif name == 'ResNet50':
        if embedding_type == "gradients":
            model = ResNet50(num_cls)
        else:
            model = models.resnet50()

    model.apply(init_weights)
    model = model.to(device)
    return model

def loss_function():
    criterion = nn.CrossEntropyLoss()
    criterion_nored = nn.CrossEntropyLoss(reduction='none')
    return criterion, criterion_nored

def optimizer_with_scheduler(model, num_epochs, learning_rate, m=0.9, wd=5e-4):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    return optimizer, scheduler

def optimizer_without_scheduler(model, learning_rate, m=0.9, wd=5e-4):
#     optimizer = optim.Adam(model.parameters(),weight_decay=wd)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    return optimizer

def generate_cumulative_timing(mod_timing):
    tmp = 0
    mod_cum_timing = np.zeros(len(mod_timing))
    for i in range(len(mod_timing)):
        tmp += mod_timing[i]
        mod_cum_timing[i] = tmp
    return mod_cum_timing/3600

def displayTable(val_err_log, tst_err_log):
    col1 = [str(i) for i in range(10)]
    val_acc = [str(100-i) for i in val_err_log]
    tst_acc = [str(100-i) for i in tst_err_log]
    table = [col1, val_acc, tst_acc]
    table = map(list, zip(*table))
    print(tabulate(table, headers=['Class', 'Val Accuracy', 'Test Accuracy'], tablefmt='orgtbl'))

def find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications, 
                       final_tst_predictions, saveDir, prefix):
    val_err_idx = list(np.where(np.array(final_val_classifications) == False)[0])
    tst_err_idx = list(np.where(np.array(final_tst_classifications) == False)[0])
    val_class_err_idxs = []
    tst_err_log = []
    val_err_log = []
    for i in range(num_cls):
        tst_class_idxs = list(torch.where(torch.Tensor(test_set.targets) == i)[0].cpu().numpy())
        val_class_idxs = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
        #err classifications per class
        val_err_class_idx = set(val_err_idx).intersection(set(val_class_idxs))
        tst_err_class_idx = set(tst_err_idx).intersection(set(tst_class_idxs))
        if(len(val_class_idxs)>0):
            val_error_perc = round((len(val_err_class_idx)/len(val_class_idxs))*100,2)
        else:
            val_error_perc = 0
        tst_error_perc = round((len(tst_err_class_idx)/len(tst_class_idxs))*100,2)
#         print("val, test error% for class ", i, " : ", val_error_perc, tst_error_perc)
        val_class_err_idxs.append(val_err_class_idx)
        tst_err_log.append(tst_error_perc)
        val_err_log.append(val_error_perc)
    displayTable(val_err_log, tst_err_log)
    tst_err_log.append(sum(tst_err_log)/len(tst_err_log))
    val_err_log.append(sum(val_err_log)/len(val_err_log))
    return tst_err_log, val_err_log, val_class_err_idxs

def aug_train_subset(train_set, lake_set, true_lake_set, subset, lake_subset_idxs, budget, augrandom=False):
    all_lake_idx = list(range(len(lake_set)))
    if(not(len(subset)==budget) and augrandom):
        print("Budget not filled, adding ", str(int(budget) - len(subset)), " randomly.")
        remain_budget = int(budget) - len(subset)
        remain_lake_idx = list(set(all_lake_idx) - set(subset))
        random_subset_idx = list(np.random.choice(np.array(remain_lake_idx), size=int(remain_budget), replace=False))
        subset += random_subset_idx
    if str(type(true_lake_set.targets)) == "<class 'numpy.ndarray'>":
        lake_ss = SubsetWithTargets(true_lake_set, subset, torch.Tensor(true_lake_set.targets.astype(np.float))[subset])
    else:
        lake_ss = SubsetWithTargets(true_lake_set, subset, torch.Tensor(true_lake_set.targets.float())[subset])
    remain_lake_idx = list(set(all_lake_idx) - set(lake_subset_idxs))
    if str(type(true_lake_set.targets)) == "<class 'numpy.ndarray'>":
        remain_lake_set = SubsetWithTargets(lake_set, remain_lake_idx, torch.Tensor(lake_set.targets.astype(np.float))[remain_lake_idx])
    else:
        remain_lake_set = SubsetWithTargets(lake_set, remain_lake_idx, torch.Tensor(lake_set.targets.float())[remain_lake_idx])
    if str(type(true_lake_set.targets)) == "<class 'numpy.ndarray'>":
        remain_true_lake_set = SubsetWithTargets(true_lake_set, remain_lake_idx, torch.Tensor(true_lake_set.targets.astype(np.float))[remain_lake_idx])
    else:
        remain_true_lake_set = SubsetWithTargets(true_lake_set, remain_lake_idx, torch.Tensor(true_lake_set.targets.float())[remain_lake_idx])
#     print(len(lake_ss),len(remain_lake_set),len(lake_set))
    aug_train_set = torch.utils.data.ConcatDataset([train_set, lake_ss])
    aug_trainloader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True, pin_memory=True)
    return aug_train_set, remain_lake_set, remain_true_lake_set, lake_ss
                        
def getQuerySet(val_set,imb_cls_idx):
    
    miscls_idx = []
    
    
    for i in imb_cls_idx:
        imb_cls_samples = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
        miscls_idx += imb_cls_samples
    print("Total samples from imbalanced classes as targets (Size of query set): ", len(miscls_idx))
    return SubsetWithTargets(val_set, miscls_idx, val_set.targets[miscls_idx])

def getPrivateSet(val_set,imb_cls_idx):
    
    miscls_idx = []
    
    
    for i in imb_cls_idx:
        imb_cls_samples = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
        miscls_idx += imb_cls_samples
    print("Total samples from imbalanced classes as targets (Size of query set): ", len(miscls_idx))
    return SubsetWithTargets(val_set, miscls_idx, val_set.targets[miscls_idx])



def getPerClassSel(lake_set, subset, num_cls):
    perClsSel = []
    if str(type(lake_set.targets)) == "<class 'numpy.ndarray'>":
        subset_cls = torch.Tensor(lake_set.targets.astype(np.float))[subset]
    else:
        subset_cls = torch.Tensor(lake_set.targets.float())[subset]
    for i in range(num_cls):
        cls_subset_idx = list(torch.where(subset_cls == i)[0].cpu().numpy())
        perClsSel.append(len(cls_subset_idx))
    return perClsSel

def print_final_results(res_dict, sel_cls_idx):
    print("Gain in overall test accuracy: ", res_dict['test_acc'][1]-res_dict['test_acc'][0])
    bf_sel_cls_acc = np.array(res_dict['all_class_acc'][0])[sel_cls_idx]
    af_sel_cls_acc = np.array(res_dict['all_class_acc'][1])[sel_cls_idx]
    print("Gain in targeted test accuracy: ", np.mean(af_sel_cls_acc-bf_sel_cls_acc))


In [ ]:
class customSampler(torch.utils.data.Sampler):
    def __init__(self, ind):
        self.ind = ind
        return
    def __iter__(self):
        return iter(self.ind)



In [10]:



def feature_extraction(inp, layer_name,model):
        feature = {}
        
        def get_features(name):
            def hook(model, inp, output):
                feature[name] = output.detach()
            return hook
        for name, layer in model._modules.items():
            if name == layer_name:
                layer.register_forward_hook(get_features(layer_name))
        output = model(inp)
        return torch.squeeze(feature[layer_name])

def get_feature_embedding(dataset, unlabeled, layer_name, model):
        dataloader = DataLoader(dataset, batch_size = 1000, shuffle = False)
        features = []
        if(unlabeled):
            for batch_idx, inputs in enumerate(dataloader):
                inputs = inputs.to(device)
                batch_features = feature_extraction(inputs, layer_name,model)
                features.append(batch_features)
        else:
            for batch_idx, (inputs,_) in enumerate(dataloader):
                inputs = inputs.to(device)
                batch_features = feature_extraction(inputs, layer_name,model)
                features.append(batch_features)
        return torch.vstack(features)



def _proj_simplex(v):
    """
    v: PyTorch Tensor to be projected to a simplex

    Returns:
    w: PyTorch Tensor simplex projection of v
    """
    z = 1
    orig_shape = v.shape
    v = v.view(1, -1)
    shape = v.shape
    with torch.no_grad():
        mu = torch.sort(v, dim=1)[0]
        mu = torch.flip(mu, dims=(1,))
        cum_sum = torch.cumsum(mu, dim=1)
        j = torch.unsqueeze(torch.arange(1, shape[1] + 1, dtype=mu.dtype, device=mu.device), 0)
        rho = torch.sum(mu * j - cum_sum + z > 0.0, dim=1, keepdim=True) - 1.
        rho = rho.to(int)
        max_nn = cum_sum[torch.arange(shape[0]), rho[:, 0]]
        theta = (torch.unsqueeze(max_nn, -1) - z) / (rho.type(max_nn.dtype) + 1)
        w = torch.clamp(v - theta, min=0.0).view(orig_shape)
        return w

# Only Target

In [11]:


# Step : Fine-tune the ResNet on the initial data until overfitting
#Create dataloaders
trainloader = torch.utils.data.DataLoader(targetset, batch_size=100,
                                              shuffle=True, pin_memory=True)


model = models.resnet50(pretrained=True)
model=model.to(device)

args = {}
model_name = 'ResNet18'
embedding_type = "features" #Type of the representation to use (gradients/features)
args['classifier_learning_rate'] = 0.0003
# Model Creation
#model = create_model(model_name, num_cls, device, embedding_type)
# Loss Functions
criterion, criterion_nored = loss_function()

# Getting the optimizer and scheduler
learning_rate = args['classifier_learning_rate']
optimizer = optimizer_without_scheduler(model, learning_rate)
full_trn_acc = 0.0
full_trn_loss=0.0
full_trn_correct = 0
full_trn_total = 0
num_ep=1
while(full_trn_acc<0.99 and num_ep<100):
            model.train()
            # if strategy != "WASSAL_Pure_Weighted":
            # if True:
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                #inputs, target = Variable(inputs), Variable(inputs)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                # print("hard loss:{}".format(loss.item()))
                loss.backward()
                optimizer.step()
            
            model.eval()
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(trainloader): #Compute Train accuracy
                    inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    full_trn_loss += loss.item()
                    _, predicted = outputs.max(1)
                    full_trn_total += targets.size(0)
                    full_trn_correct += predicted.eq(targets).sum().item()
                full_trn_acc = full_trn_correct / full_trn_total
                print(" Training epoch [" , num_ep, "]" , " Training Acc: ", full_trn_acc, end="\r")
                num_ep+=1

loss_func = SamplesLoss("sinkhorn", p=2, blur=0.1, scaling=0.1)

unlabeled_dataset_len=len(lake_set)
shuffled_indices = list(range(unlabeled_dataset_len))
random.shuffle(shuffled_indices)
sampler = customSampler(shuffled_indices)

query_dataset = getQuerySet(val_set,split_cfg['sel_cls_idx'])
query_dataset_len = len(query_dataset)

refrain_dataset = getPrivateSet(val_set,split_cfg['sel_cls_idx'])
refrain_dataset_len = len(refrain_dataset)

minibatch_size = 500
step_size=10
num_batches = math.ceil(unlabeled_dataset_len/minibatch_size)


 #uniform distribution of weights
simplex_query= Variable(torch.ones(unlabeled_dataset_len, requires_grad=True, device=device)/unlabeled_dataset_len)
simplex_refrain = Variable(torch.ones(unlabeled_dataset_len, requires_grad=True, device=device)/unlabeled_dataset_len) 
beta = torch.ones(query_dataset_len, requires_grad=False)/query_dataset_len
gamma = torch.ones(refrain_dataset_len, requires_grad=False)/refrain_dataset_len
unlabeled_dataloader = DataLoader(dataset=lake_set, batch_size=minibatch_size, shuffle=False, sampler=sampler)
query_dataloader = DataLoader(dataset=query_dataset, batch_size=len(query_dataset), shuffle=False)
refrain_dataloader = DataLoader(dataset=refrain_dataset, batch_size=len(refrain_dataset), shuffle=False)

query_iter=iter(query_dataloader)
refrain_iter=iter(refrain_dataloader)

query_imgs, _= next(query_iter)
query_imgs=query_imgs.to(device)
refrain_imgs, _= next(refrain_iter)
refrain_imgs=refrain_imgs.to(device)

beta = beta.to(device)
gamma = gamma.to(device)


# Hyperparameters
lr = 0.1
step_size = 5
optimizer = torch.optim.Adam([simplex_query,beta], lr=lr)

# Define the learning rate scheduler
scheduler_query = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)
simplex_query.requires_grad = True
#beta.requires_grad = True



# Create lists to store the loss values
        
overall_loss=[]
        
# Loop over the datasets 10 times
for i in range(100):
    
    simplex_query.grad = None  # Reset gradients at the beginning of each epoch
    batch_idx = 0
    # Initialize loss_avg as a tensor with requires_grad=True
    loss_avg = torch.tensor(0.0, requires_grad=True)
   
    optimizer.zero_grad()
    #batchwise WD calculation
    for unlabeled_imgs,_ in unlabeled_dataloader:
         # Don't calculate gradients
        with torch.no_grad():     
            unlabeled_features = get_feature_embedding(unlabeled_imgs, True, 'avgpool',model)
            query_features = get_feature_embedding(query_imgs, True, 'avgpool',model)
            refrain_features = get_feature_embedding(refrain_imgs, True, 'avgpool',model)
                    
        # Flatten the features to be 1D tensors
        unlabeled_features = unlabeled_features.view(unlabeled_features.shape[0], -1)
        query_features = query_features.view(query_features.shape[0], -1) 
        refrain_features = refrain_features.view(refrain_features.shape[0], -1)

        simplex_batch_query = simplex_query[batch_idx * unlabeled_dataloader.batch_size : (batch_idx + 1) * unlabeled_dataloader.batch_size]
        
        #TODO: should we average or project?
        simplex_batch_query = simplex_batch_query.clone() / simplex_batch_query.sum()
        simplex_batch_query=simplex_batch_query.to(device)

        
        
        simplex_batch_refrain = simplex_refrain[batch_idx * unlabeled_dataloader.batch_size : (batch_idx + 1) * unlabeled_dataloader.batch_size]
        simplex_batch_refrain = simplex_batch_refrain.clone() / simplex_batch_refrain.sum()
        simplex_batch_refrain=simplex_batch_refrain.to(device)

        
       
       
        loss = loss_func(simplex_batch_query, unlabeled_features, beta, query_features)

        overall_loss.append(loss.item())
                
        loss_avg = loss_avg + loss / num_batches
        #loss_avg=0.5*loss_avg
        batch_idx += 1
        print("Batchwise loss: {}".format(loss), end="\r")
            
    loss_avg.backward()
    optimizer.step()
   
    scheduler_query.step()
   

    with torch.no_grad():
        simplex_query.data = _proj_simplex(simplex_query.data)
   
    print("Epoch:[]", i, "]. Avg loss: [{}]".format(loss_avg), end="\r")

    import matplotlib.pyplot as plt
    import numpy as np

    # Assuming your unlabeled_dataset is a PyTorch Dataset that returns a tuple of (image, class) 
    # and that 'classes' is a list of class labels for each image in the dataset

    classes = np.array([label for _, label in lake_set])

    # Converting the simplex_target tensor to numpy array for plotting
    simplex_np = simplex_query.detach().cpu().numpy()

    # Setting up the y-values to be 1 since it's a one-dimensional plot
    y = classes

    # create a color map to color your data points based on their class
    colors = plt.cm.rainbow(np.linspace(0, 1, 2))  # 10 distinct colors for 10 classes

    fig, ax = plt.subplots()

    for i in range(0, 2):  # assuming classes range from 1 to 10
        ax.scatter(simplex_np[classes==i], y[classes==i], color=colors[i-1], label='Class '+str(i))

    ax.set_yticks([])  # remove the y-axis
    ax.legend(loc='upper right')

    plt.show()
    

NameError: name 'targetset' is not defined

In [12]:


# Step : Fine-tune the ResNet on the initial data until overfitting
#Create dataloaders
trainloader = torch.utils.data.DataLoader(targetset, batch_size=100,
                                              shuffle=True, pin_memory=True)


#model = models.resnet50(pretrained=True)
#model=model.to(device)

args = {}
model_name = 'ResNet18'
embedding_type = "features" #Type of the representation to use (gradients/features)
args['classifier_learning_rate'] = 0.0003
# Model Creation
model = create_model(model_name, num_cls, device, embedding_type)
# Loss Functions
criterion, criterion_nored = loss_function()

# Getting the optimizer and scheduler
learning_rate = args['classifier_learning_rate']
optimizer = optimizer_without_scheduler(model, learning_rate)
full_trn_acc = 0.0
full_trn_loss=0.0
full_trn_correct = 0
full_trn_total = 0
num_ep=1
while(full_trn_acc<0.99 and num_ep<100):
            model.train()
            # if strategy != "WASSAL_Pure_Weighted":
            # if True:
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                inputs, target = Variable(inputs), Variable(inputs)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                # print("hard loss:{}".format(loss.item()))
                loss.backward()
                optimizer.step()
            
            model.eval()
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(trainloader): #Compute Train accuracy
                    inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    full_trn_loss += loss.item()
                    _, predicted = outputs.max(1)
                    full_trn_total += targets.size(0)
                    full_trn_correct += predicted.eq(targets).sum().item()
                full_trn_acc = full_trn_correct / full_trn_total
                print(" Training epoch [" , num_ep, "]" , " Training Acc: ", full_trn_acc, end="\r")
                num_ep+=1

loss_func = SamplesLoss("sinkhorn", p=2, blur=0.1, scaling=0.1)

unlabeled_dataset_len=len(lake_set)
shuffled_indices = list(range(unlabeled_dataset_len))
random.shuffle(shuffled_indices)
sampler = customSampler(shuffled_indices)

query_dataset = getQuerySet(trainset,split_cfg['sel_cls_idx'])
query_dataset_len = len(query_dataset)

private_dataset = getPrivateSet(trainset,split_cfg['sel_cls_idx'])
private_dataset_len = len(private_dataset)

minibatch_size = 500
step_size=10
num_batches = math.ceil(unlabeled_dataset_len/minibatch_size)


 #uniform distribution of weights
simplex_target= Variable(torch.ones(unlabeled_dataset_len, requires_grad=True, device=device)/unlabeled_dataset_len)
simplex_refrain = Variable(torch.ones(unlabeled_dataset_len, requires_grad=True, device=device)/unlabeled_dataset_len) 
beta = torch.ones(query_dataset_len, requires_grad=False)/query_dataset_len
gamma = torch.ones(private_dataset_len, requires_grad=False)/private_dataset_len
unlabeled_dataloader = DataLoader(dataset=lake_set, batch_size=minibatch_size, shuffle=False, sampler=sampler)
target_dataloader = DataLoader(dataset=query_dataset, batch_size=len(query_dataset), shuffle=False)
private_dataloader = DataLoader(dataset=private_dataset, batch_size=len(private_dataset), shuffle=False)

target_iter=iter(target_dataloader)
private_iter=iter(private_dataloader)

target_imgs, _= next(target_iter)
target_imgs=target_imgs.to(device)
private_imgs, _= next(private_iter)
private_imgs=private_imgs.to(device)

beta = beta.to(device)
gamma = gamma.to(device)


# Hyperparameters
lr = 0.001
step_size = 40
optimizer = torch.optim.Adam([simplex_target,beta], lr=lr)
optimizer_refrain = torch.optim.Adam([simplex_refrain,gamma], lr=lr)

# Define the learning rate scheduler
scheduler_target = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)
scheduler_refrain = torch.optim.lr_scheduler.StepLR(optimizer_refrain, step_size=step_size, gamma=0.1)
simplex_target.requires_grad = True
simplex_refrain.requires_grad = True  


# Create lists to store the loss values
        
overall_loss=[]
        
# Loop over the datasets 10 times
for i in range(100):
    
    simplex_target.grad = None  # Reset gradients at the beginning of each epoch
    batch_idx = 0
    # Initialize loss_avg as a tensor with requires_grad=True
    loss_avg = torch.tensor(0.0, requires_grad=True)
   
    optimizer.zero_grad()
    #batchwise WD calculation
    for unlabeled_imgs,_ in unlabeled_dataloader:
         # Don't calculate gradients
        with torch.no_grad():     
            unlabeled_features = get_feature_embedding(unlabeled_imgs, True, 'avgpool',model)
            target_features = get_feature_embedding(target_imgs, True, 'avgpool',model)
            private_features = get_feature_embedding(private_imgs, True, 'avgpool',model)
                    
        # Flatten the features to be 1D tensors
        unlabeled_features = unlabeled_features.view(unlabeled_features.shape[0], -1)
        target_features = target_features.view(target_features.shape[0], -1) 
        private_features = private_features.view(private_features.shape[0], -1)

        simplex_batch_target = simplex_target[batch_idx * unlabeled_dataloader.batch_size : (batch_idx + 1) * unlabeled_dataloader.batch_size]
        
        #TODO: should we average or project?
        simplex_batch_target = simplex_batch_target.clone() / simplex_batch_target.sum()
        simplex_batch_target=simplex_batch_target.to(device)

        
        
        simplex_batch_refrain = simplex_refrain[batch_idx * unlabeled_dataloader.batch_size : (batch_idx + 1) * unlabeled_dataloader.batch_size]
        simplex_batch_refrain = simplex_batch_refrain.clone() / simplex_batch_refrain.sum()
        simplex_batch_refrain=simplex_batch_refrain.to(device)

        
        l1 = loss_func(simplex_batch_target, unlabeled_features, beta, target_features)
        l2 = loss_func(simplex_batch_target, unlabeled_features, gamma, private_features)
        l3 = loss_func(simplex_batch_target, unlabeled_features, simplex_batch_refrain, unlabeled_features)
        loss = (1*l1) + (1*l2) - (0.3*l3)

        overall_loss.append(loss.item())
                
        loss_avg = loss_avg + loss / num_batches
        #loss_avg=0.5*loss_avg
        batch_idx += 1
        print("Batchwise loss: {}".format(loss), end="\r")
            
    loss_avg.backward()
    optimizer.step()
    optimizer_refrain.step()
    scheduler_target.step()
    scheduler_refrain.step()

    with torch.no_grad():
        simplex_target.data = _proj_simplex(simplex_target.data)
   
    print("Epoch:[]", i, "]. Avg loss: [{}]".format(loss_avg), end="\r")

    import matplotlib.pyplot as plt
    import numpy as np

    # Assuming your unlabeled_dataset is a PyTorch Dataset that returns a tuple of (image, class) 
    # and that 'classes' is a list of class labels for each image in the dataset

    classes = np.array([label for _, label in lake_set])

    # Converting the simplex_target tensor to numpy array for plotting
    simplex_np = simplex_target.detach().cpu().numpy()

    # Setting up the y-values to be 1 since it's a one-dimensional plot
    y = np.ones_like(simplex_np)

    # create a color map to color your data points based on their class
    colors = plt.cm.rainbow(np.linspace(0, 1, 2))  # 10 distinct colors for 10 classes

    fig, ax = plt.subplots()

    for i in range(0, 2):  # assuming classes range from 1 to 10
        ax.scatter(simplex_np[classes==i], y[classes==i], color=colors[i-1], label='Class '+str(i))

    ax.set_yticks([])  # remove the y-axis
    ax.legend(loc='upper right')

    plt.show()

NameError: name 'targetset' is not defined

In [ ]:
len(simplex_query)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming your unlabeled_dataset is a PyTorch Dataset that returns a tuple of (image, class) 
# and that 'classes' is a list of class labels for each image in the dataset

classes = np.array([label for _, label in lake_set])

# Converting the simplex_target tensor to numpy array for plotting
simplex_np = simplex_refrain.detach().cpu().numpy()

# Setting up the y-values to be 1 since it's a one-dimensional plot
y = classes

# create a color map to color your data points based on their class
colors = plt.cm.rainbow(np.linspace(0, 1, 2))  # 10 distinct colors for 10 classes

fig, ax = plt.subplots()

for i in range(0, 2):  # assuming classes range from 1 to 10
    ax.scatter(simplex_np[classes==i], y[classes==i], color=colors[i-1], label='Class '+str(i))

ax.set_yticks([])  # remove the y-axis
ax.legend(loc='upper right')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Assuming you have a way to get the class labels for your unlabeled_dataset
classes = np.array([label for _, label in unlabeled_dataset])  # replace with your method of getting labels

# Converting the simplex_target tensor to numpy array for plotting
simplex_np = simplex_target.detach().cpu().numpy()

# create a color map to color your data points based on their class
colors = plt.cm.rainbow(np.linspace(0, 1, 10))  # 10 distinct colors for 10 classes

fig, ax = plt.subplots()

for i in range(1, 11):  # assuming classes range from 1 to 10
    indices = np.where(classes == i)
    sns.distplot(simplex_np[indices], color=colors[i-1], label='Class '+str(i), ax=ax)

ax.legend(loc='upper right')

plt.show()


In [ ]:
colors